# Setup Ambiente e Repository

## Clonazione e Configurazione Branch 'Marco'

In [2]:
import os
import shutil
import subprocess
import sys

# Reset Posizione Base
os.chdir("/content")
project_path = "/content/Visual-Place-Recognition-Project"

# Pulizia preventiva
if os.path.exists(project_path):
    print(f"🧹 Pulizia vecchia cartella...")
    shutil.rmtree(project_path)

# Clonazione Repository (Versione Nikcian)
print("📥 Clonazione del repository (Nikcian)...")
subprocess.run(["git", "clone", "https://github.com/nikcian/Visual-Place-Recognition-Project.git"], check=True)

if os.path.exists(project_path):
    os.chdir(project_path)
    print(f"📂 Entrato in: {os.getcwd()}")

    print("🔀 Passaggio al branch 'Marco'...")
    res = subprocess.run(["git", "checkout", "Marco"], capture_output=True)
    if res.returncode != 0:
        print("⚠️ Branch remoto non trovato, creo locale da main...")
        subprocess.run(["git", "checkout", "-b", "Marco"])

📥 Clonazione del repository (Nikcian)...
📂 Entrato in: /content/Visual-Place-Recognition-Project
🔀 Passaggio al branch 'Marco'...


## Installazione Dipendenze e Fix Librerie

In [3]:
# FIX REQUIREMENTS
req_path = os.path.join("VPR-methods-evaluation", "requirements.txt")
new_req_path = os.path.join("VPR-methods-evaluation", "requirements_colab.txt")

if os.path.exists(req_path):
    print(f"🔧 Adattamento requirements per Colab...")
    with open(req_path, "r") as f:
        lines = f.readlines()
    with open(new_req_path, "w") as f:
        for line in lines:
            pkg = line.split("==")[0].split(">=")[0].strip()
            if pkg.lower() in ["torch", "torchvision", "numpy", "matplotlib", "scipy", "pillow"]:
                continue
            if pkg: f.write(f"{pkg}\n")

    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", new_req_path, "--quiet"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-gpu", "gdown", "loguru", "einops", "--quiet"])
    print("🎉 Setup base completato!")

# Riparazione manuale e gestione FAISS
print("⚙️ Configurazione Faiss e utility...")
for pkg in ["gdown", "loguru", "einops"]:
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "--quiet"])

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-gpu", "--quiet"])
    print("✅ Faiss-GPU installato.")
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faiss-cpu", "--quiet"])
    print("✅ Fallback: Faiss-CPU installato.")

🔧 Adattamento requirements per Colab...


CalledProcessError: Command '['/usr/bin/python3', '-m', 'pip', 'install', 'faiss-gpu', 'gdown', 'loguru', 'einops', '--quiet']' returned non-zero exit status 1.

In [4]:
import subprocess
import sys

print("🔧 Tentativo di riparazione manuale delle librerie...")

def install(package):
    try:
        print(f"📦 Installazione {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"✅ {package} installato!")
        return True
    except subprocess.CalledProcessError:
        print(f"⚠️ Errore installando {package}.")
        return False

# 1. Installiamo le librerie accessorie sicure
install("gdown")
install("loguru")
install("einops")

# 2. Gestione Intelligente di FAISS
print("⚙️ Configurazione Faiss...")
# Prima proviamo la versione GPU
if not install("faiss-gpu"):
    print("🔄 Fallback: Provo a installare faiss-cpu...")
    # Se fallisce, proviamo la CPU
    if install("faiss-cpu"):
        print("✅ Faiss-CPU installato con successo (modalità compatibile).")
    else:
        print("❌ Errore critico: Impossibile installare Faiss (né GPU né CPU).")
else:
    print("✅ Faiss-GPU installato correttamente.")

print("\n🎉 Riparazione completata. Ora puoi procedere al download dei dati.")

🔧 Tentativo di riparazione manuale delle librerie...
📦 Installazione gdown...
✅ gdown installato!
📦 Installazione loguru...
✅ loguru installato!
📦 Installazione einops...
✅ einops installato!
⚙️ Configurazione Faiss...
📦 Installazione faiss-gpu...
⚠️ Errore installando faiss-gpu.
🔄 Fallback: Provo a installare faiss-cpu...
📦 Installazione faiss-cpu...
✅ faiss-cpu installato!
✅ Faiss-CPU installato con successo (modalità compatibile).

🎉 Riparazione completata. Ora puoi procedere al download dei dati.


# Gestione Dataset e Modelli Esterni

## Download Dataset (SF, Tokyo, SVOX)

In [5]:
import os
import gdown
import shutil

os.chdir(project_path)
os.makedirs("data", exist_ok=True)

datasets = {
    "sf_xs": "https://drive.google.com/file/d/1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y/view?usp=share_link",
    "tokyo_xs": "https://drive.google.com/file/d/15QB3VNKj93027UAQWv7pzFQO1JDCdZj2/view?usp=share_link",
    "svox": "https://drive.google.com/file/d/16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9/view?usp=drive_link"
}

for name, url in datasets.items():
    if not os.path.exists(f"data/{name}"):
        print(f"⬇️ Scaricando {name}...")
        gdown.download(url, f"data/{name}.zip", quiet=False, fuzzy=True)
        shutil.unpack_archive(f"data/{name}.zip", "data")
        os.remove(f"data/{name}.zip")

⬇️ Scaricando sf_xs...


Downloading...
From (original): https://drive.google.com/uc?id=1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y
From (redirected): https://drive.google.com/uc?id=1tQqEyt3go3vMh4fj_LZrRcahoTbzzH-y&confirm=t&uuid=d27335e4-08da-4252-867b-e6219726311e
To: /content/Visual-Place-Recognition-Project/data/sf_xs.zip
100%|██████████| 1.03G/1.03G [00:11<00:00, 89.7MB/s]


⬇️ Scaricando tokyo_xs...


Downloading...
From (original): https://drive.google.com/uc?id=15QB3VNKj93027UAQWv7pzFQO1JDCdZj2
From (redirected): https://drive.google.com/uc?id=15QB3VNKj93027UAQWv7pzFQO1JDCdZj2&confirm=t&uuid=31387073-b94c-41c5-8eca-ad3e840190cf
To: /content/Visual-Place-Recognition-Project/data/tokyo_xs.zip
100%|██████████| 141M/141M [00:01<00:00, 123MB/s]


⬇️ Scaricando svox...


Downloading...
From (original): https://drive.google.com/uc?id=16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9
From (redirected): https://drive.google.com/uc?id=16iuk8voW65GaywNUQlWAbDt6HZzAJ_t9&confirm=t&uuid=8283d7ac-d9eb-45cc-b7a7-2fc77ce6d7d6
To: /content/Visual-Place-Recognition-Project/data/svox.zip
100%|██████████| 3.51G/3.51G [00:49<00:00, 70.3MB/s]


## Setup Image Matching Models

In [6]:
# Fix cartella esterna e sottomoduli
target_folder = "image-matching-models"
if os.path.exists(target_folder): shutil.rmtree(target_folder)
subprocess.run(["git", "clone", "--recursive", "https://github.com/alexstoken/image-matching-models.git", target_folder], check=True)

# Installazione modelli
%cd {target_folder}
!pip install -e . --quiet
!pip install git+https://github.com/cvg/LightGlue.git yacs kornia kornia_moons py3_wget --quiet
%cd ..

# Configurazione PYTHONPATH per SuperGlue
cwd = os.getcwd()
immatch_path = f"{cwd}/image-matching-models/matching/third_party/imatch-toolbox"
patch2pix_path = f"{immatch_path}/third_party/patch2pix"
sys.path.insert(0, immatch_path)
sys.path.insert(0, patch2pix_path)
os.environ["PYTHONPATH"] = f"{immatch_path}:{patch2pix_path}:" + os.environ.get("PYTHONPATH", "")

/content/Visual-Place-Recognition-Project/image-matching-models
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

# Generazione File TXT (da preds.npy)

## Recupero e Generazione per San Francisco XS e Tokyo XS

In [15]:
import shutil
import os
from google.colab import drive

# 1. Rimuove forzatamente la cartella se esiste e non è montata bene
if os.path.exists('/content/drive'):
    try:
        shutil.rmtree('/content/drive')
    except:
        print("Cartella drive occupata, procedo al mount forzato...")

# 2. Riesegui il mount (clicca sul link o autorizza il pop-up)
drive.mount('/content/drive', force_remount=True)

# 3. Verifica finale: ora dovresti vedere le tue cartelle
print("\n✅ Cartelle trovate nel Drive:")
print(os.listdir('/content/drive/MyDrive/'))

Mounted at /content/drive

✅ Cartelle trovate nel Drive:
['Colab Notebooks', 'tokyo_xs.zip', 'VPR_Risultati_Finali', 'sf_xs.zip', 'svox.zip', 'VPR_Project_Final_Results']


In [16]:
import os
import numpy as np
import glob
from pathlib import Path
from google.colab import drive

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive', force_remount=True)

def process_dataset_to_txt(name, folder_on_drive):
    print(f"🔍 Elaborazione {name}...")
    output_dir = f"/content/Visual-Place-Recognition-Project/logs/netvlad_{name}/predictions_txt"
    os.makedirs(output_dir, exist_ok=True)

    # Percorso basato sulla tua struttura comunicata
    drive_root = "/content/drive/MyDrive/VPR_Risultati_Finali"
    target_path = os.path.join(drive_root, folder_on_drive, "data/preds/preds.npy")

    if os.path.exists(target_path):
        print(f"✅ TROVATO in: {target_path}")
        preds = np.load(target_path)
    else:
        # Fallback: ricerca ricorsiva se il percorso fisso fallisce
        print(f"⚠️ Percorso fisso non trovato. Avvio ricerca ricorsiva per {folder_on_drive}...")
        found_npy = next((p for p in Path(drive_root).rglob("preds.npy")
                         if folder_on_drive in str(p).lower()), None)
        if not found_npy:
            return print(f"❌ ERRORE: preds.npy non trovato per {name} in {drive_root}")
        preds = np.load(found_npy)

    # Percorsi immagini locali
    db_folder = f"/content/Visual-Place-Recognition-Project/data/{name}/test/database"
    q_folder = f"/content/Visual-Place-Recognition-Project/data/{name}/test/queries"

    db_paths = sorted(glob.glob(f"{db_folder}/*.jpg") + glob.glob(f"{db_folder}/*.png"))
    q_paths = sorted(glob.glob(f"{q_folder}/*.jpg") + glob.glob(f"{q_folder}/*.png"))

    for i, row in enumerate(preds):
        if i >= len(q_paths): break
        out_file = os.path.join(output_dir, f"{i}.txt")
        with open(out_file, "w") as f:
            f.write(f"Query: {os.path.basename(q_paths[i])}\n{os.path.abspath(q_paths[i])}\nPredictions:\n")
            for db_idx in row:
                if db_idx < len(db_paths):
                    f.write(f"{os.path.abspath(db_paths[int(db_idx)])}\n")

    print(f"🎉 Creati {len(os.listdir(output_dir))} file TXT in: {output_dir}")

# Esecuzione con i nomi cartella corretti del tuo Drive
process_dataset_to_txt("sf_xs", "netvlad_sfxs")
process_dataset_to_txt("tokyo_xs", "netvlad_tokyo")

🔍 Elaborazione sf_xs...
⚠️ Percorso fisso non trovato. Avvio ricerca ricorsiva per netvlad_sfxs...
🎉 Creati 1000 file TXT in: /content/Visual-Place-Recognition-Project/logs/netvlad_sf_xs/predictions_txt
🔍 Elaborazione tokyo_xs...
⚠️ Percorso fisso non trovato. Avvio ricerca ricorsiva per netvlad_tokyo...
🎉 Creati 315 file TXT in: /content/Visual-Place-Recognition-Project/logs/netvlad_tokyo_xs/predictions_txt


## Generazione per SVOX (Sun & Night)

In [17]:
def generate_svox_txt(condition):
    name = f"svox_{condition}"
    folder_on_drive = f"netvlad_svox_{condition}"
    print(f"🔍 Elaborazione {name.upper()}...")

    output_dir = f"/content/Visual-Place-Recognition-Project/logs/{folder_on_drive}/predictions_txt"
    os.makedirs(output_dir, exist_ok=True)

    drive_root = "/content/drive/MyDrive/VPR_Risultati_Finali"
    # Struttura specifica per SVOX: netvlad_svox_sun -> data -> preds -> preds.npy
    target_path = os.path.join(drive_root, folder_on_drive, "data/preds/preds.npy")

    if os.path.exists(target_path):
        preds = np.load(target_path)
    else:
        found_npy = next((p for p in Path(drive_root).rglob("preds.npy")
                         if folder_on_drive in str(p).lower()), None)
        if not found_npy: return print(f"❌ ERRORE: Preds non trovato per {name}")
        preds = np.load(found_npy)

    # Immagini SVOX
    q_path_obj = list(Path("/content/Visual-Place-Recognition-Project/data/svox").rglob(f"queries_{condition}"))
    if not q_path_obj: return print(f"❌ ERRORE: Immagini SVOX {condition} non trovate.")

    q_folder = str(q_path_obj[0])
    db_folder = os.path.join(os.path.dirname(q_folder), "gallery")

    db_paths = sorted(glob.glob(f"{db_folder}/*.jpg") + glob.glob(f"{db_folder}/*.png"))
    q_paths = sorted(glob.glob(f"{q_folder}/*.jpg") + glob.glob(f"{q_folder}/*.png"))

    for i, row in enumerate(preds):
        if i >= len(q_paths): break
        out_file = os.path.join(output_dir, f"{i}.txt")
        with open(out_file, "w") as f:
            f.write(f"Query: {os.path.basename(q_paths[i])}\n{os.path.abspath(q_paths[i])}\nPredictions:\n")
            for db_idx in row:
                if db_idx < len(db_paths):
                    f.write(f"{os.path.abspath(db_paths[int(db_idx)])}\n")
    print(f"✅ TXT Generati per {name}")

generate_svox_txt("sun")
generate_svox_txt("night")

🔍 Elaborazione SVOX_SUN...
✅ TXT Generati per svox_sun
🔍 Elaborazione SVOX_NIGHT...
✅ TXT Generati per svox_night


# Esecuzione Matching

## sf_xs

In [1]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import shutil

# --- 1. CONFIGURAZIONE PERCORSI ---
dataset_name = "sf_xs"
matchers = ["superpoint-lg", "loftr", "superglue"]
project_root = "/content/Visual-Place-Recognition-Project"
%cd {project_root}

# Cartelle su Drive per il salvataggio permanente
drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset_name}"
csv_drive_dir = f"{drive_base}/CSVs"
torch_drive_dir = f"{drive_base}/torch_files"

os.makedirs(csv_drive_dir, exist_ok=True)
os.makedirs(torch_drive_dir, exist_ok=True)

# --- 2. FUNZIONE ESTRAZIONE CSV (Adattata al tuo codice) ---
def save_inliers_csv(preds_dir, matcher_name):
    print(f"📄 Generazione CSV per {matcher_name}...")
    inliers_dir = Path(f"{preds_dir}_{matcher_name}")
    data_list = []

    # Cerchiamo tutti i .torch generati
    torch_files = sorted(list(inliers_dir.glob("*.torch")), key=lambda x: int(x.stem))

    for t_file in torch_files:
        # Carichiamo la lista di risultati (uno per ogni candidato top-20)
        results = torch.load(t_file, weights_only=False)
        # Prendiamo il numero di inliers del PRIMO candidato (quello che determina R@1)
        # o salviamo la media/max se preferisci per le estensioni 6.1/6.2
        num_inliers_top1 = results[0]['num_inliers'] if len(results) > 0 else 0

        data_list.append({
            'query_id': t_file.stem,
            'num_inliers': num_inliers_top1
        })

    df = pd.DataFrame(data_list)
    csv_path = f"{csv_drive_dir}/{matcher_name}_inliers.csv"
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV salvato: {csv_path}")

# --- 3. ESECUZIONE BATCH ---
for matcher in matchers:
    print(f"\n{'-'*50}\n🔥 MATCHING CORRENTE: {matcher.upper()}\n{'-'*50}")

    preds_dir = f"logs/netvlad_{dataset_name}/predictions_txt"
    inliers_local_dir = f"{preds_dir}_{matcher}"

    # A. Esecuzione Matching
    !python match_queries_preds.py \
        --preds-dir "$preds_dir" \
        --matcher "$matcher" \
        --device cuda \
        --im-size 512 \
        --num-preds 20

    # B. Backup dei .torch su Drive (Necessario per non perderli)
    print(f"📦 Backup file .torch su Drive...")
    dest_torch = f"{torch_drive_dir}/{matcher}"
    if os.path.exists(dest_torch): shutil.rmtree(dest_torch)
    shutil.copytree(inliers_local_dir, dest_torch)

    # C. Generazione CSV per Estensioni
    save_inliers_csv(preds_dir, matcher)

    # D. Esecuzione Reranking (Metrica Recall)
    print(f"📈 Calcolo Recall...")
    !python reranking.py \
        --preds-dir "$preds_dir" \
        --inliers-dir "$inliers_local_dir" \
        --num-preds 20

print(f"\n{'-'*50}\n🎉 PROCESSO COMPLETATO PER SF-XS\n{'-'*50}")

[Errno 2] No such file or directory: '/content/Visual-Place-Recognition-Project'
/content

--------------------------------------------------
🔥 MATCHING CORRENTE: SUPERPOINT-LG
--------------------------------------------------
python3: can't open file '/content/match_queries_preds.py': [Errno 2] No such file or directory
📦 Backup file .torch su Drive...


FileNotFoundError: [Errno 2] No such file or directory: 'logs/netvlad_sf_xs/predictions_txt_superpoint-lg'

In [18]:
import os
import shutil
from pathlib import Path

# --- 1. CONFIGURAZIONE ---
dataset_name = "sf_xs"
# Riprendiamo LoFTR (che salterà i file esistenti) e poi SuperGlue
matchers = ["loftr", "superglue"]
project_root = "/content/Visual-Place-Recognition-Project"
%cd {project_root}

# Percorsi Drive per salvataggio permanente
drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset_name}"
csv_drive_dir = f"{drive_base}/CSVs"
torch_drive_dir = f"{drive_base}/torch_files"

os.makedirs(csv_drive_dir, exist_ok=True)
os.makedirs(torch_drive_dir, exist_ok=True)

# --- 2. FUNZIONE ESTRAZIONE CSV (Semplice, come la precedente) ---
def save_inliers_csv_simple(preds_dir, matcher_name):
    print(f"📄 Generazione CSV temporaneo per {matcher_name}...")
    inliers_dir = Path(f"{preds_dir}_{matcher_name}")
    data_list = []

    torch_files = sorted(list(inliers_dir.glob("*.torch")), key=lambda x: int(x.stem))
    for t_file in torch_files:
        results = torch.load(t_file, weights_only=False)
        num_inliers_top1 = results[0]['num_inliers'] if len(results) > 0 else 0
        data_list.append({'query_id': t_file.stem, 'num_inliers': num_inliers_top1})

    df = pd.DataFrame(data_list)
    csv_path = f"{csv_drive_dir}/{matcher_name}_inliers.csv"
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV salvato: {csv_path}")

# --- 3. ESECUZIONE BATCH ---
for matcher in matchers:
    print(f"\n{'-'*50}\n🔥 RIPRESA MATCHING: {matcher.upper()}\n{'-'*50}")

    preds_dir = f"logs/netvlad_{dataset_name}/predictions_txt"
    inliers_local_dir = f"{preds_dir}_{matcher}"

    # A. Esecuzione Matching (Riprende dal punto di interruzione)
    !python match_queries_preds.py \
        --preds-dir "$preds_dir" \
        --matcher "$matcher" \
        --device cuda \
        --im-size 512 \
        --num-preds 20

    # B. Backup dei .torch su Drive (Fondamentale!)
    print(f"📦 Backup file .torch su Drive...")
    dest_torch = f"{torch_drive_dir}/{matcher}"
    if os.path.exists(dest_torch): shutil.rmtree(dest_torch)
    shutil.copytree(inliers_local_dir, dest_torch)

    # C. CSV Semplice
    save_inliers_csv_simple(preds_dir, matcher)

    # D. Reranking Base
    print(f"📈 Calcolo Recall base...")
    !python reranking.py \
        --preds-dir "$preds_dir" \
        --inliers-dir "$inliers_local_dir" \
        --num-preds 20

print(f"\n🎉 SF-XS COMPLETATO. I file .torch sono al sicuro su Drive.")

/content/Visual-Place-Recognition-Project

--------------------------------------------------
🔥 RIPRESA MATCHING: LOFTR
--------------------------------------------------
Downloading: "http://cmp.felk.cvut.cz/~mishkdmy/models/loftr_outdoor.ckpt" to /root/.cache/torch/hub/checkpoints/loftr_outdoor.ckpt
100% 44.2M/44.2M [00:03<00:00, 13.9MB/s]
100% 1000/1000 [1:02:51<00:00,  3.77s/it]
📦 Backup file .torch su Drive...
📄 Generazione CSV temporaneo per loftr...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/loftr_inliers.csv
📈 Calcolo Recall base...
100% 1000/1000 [00:01<00:00, 617.88it/s]
R@1: 53.6, R@5: 55.3, R@10: 55.8, R@20: 56.2, R@100: 56.2

--------------------------------------------------
🔥 RIPRESA MATCHING: SUPERGLUE
--------------------------------------------------
/content/Visual-Place-Recognition-Project/image-matching-models/matching/third_party/imatch-toolbox/third_party/patch2pix/networks/ncn/model.py:233: SyntaxWarning: "is not" with 'str' liter

### Generazione File CSV corretto

In [19]:
import os
import torch
import pandas as pd
from pathlib import Path
from util import get_list_distances_from_preds

# Configurazione (Assicurati che Drive sia montato)
dataset_name = "sf_xs"
matchers = ["superpoint-lg", "loftr", "superglue"]
threshold = 25  # Soglia metri definita dal progetto

# Percorsi coerenti con il tuo script batch
drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset_name}"
csv_drive_dir = f"{drive_base}/CSVs"
torch_drive_dir = f"{drive_base}/torch_files"
preds_dir = f"/content/Visual-Place-Recognition-Project/logs/netvlad_{dataset_name}/predictions_txt"

def enrich_csv_from_drive():
    for matcher in matchers:
        print(f"\n📊 Arricchimento dati per: {matcher.upper()}")

        # Percorso dei .torch su Drive (già backuppati)
        matcher_torch_path = Path(f"{torch_drive_dir}/{matcher}")

        if not matcher_torch_path.exists():
            print(f"⚠️ Cartella torch non trovata su Drive per {matcher}. Salto...")
            continue

        data_list = []
        torch_files = sorted(list(matcher_torch_path.glob("*.torch")), key=lambda x: int(x.stem))

        for t_file in torch_files:
            # 1. Carica distanze reali per determinare is_correct
            txt_file = f"{preds_dir}/{t_file.stem}.txt"
            # get_list_distances_from_preds restituisce le distanze GPS originali
            geo_dists = get_list_distances_from_preds(txt_file)

            # 2. Carica i risultati del matching
            query_results = torch.load(t_file, weights_only=False)

            # 3. Trova il massimo numero di inliers tra i 20 candidati (per ordinamento)
            inliers_counts = [res['num_inliers'] for res in query_results]
            max_val = max(inliers_counts) if inliers_counts else 0

            # 4. Trova l'indice del migliore per determinare se è corretto (R@1 reranked)
            best_idx = inliers_counts.index(max_val) if inliers_counts else 0
            is_correct = 1 if (geo_dists[best_idx] <= threshold) else 0

            data_list.append({
                'query_id': t_file.stem,
                'max_inliers': max_val,
                'is_correct': is_correct
            })

        # Generazione e sovrascrittura CSV
        df = pd.DataFrame(data_list)
        csv_path = f"{csv_drive_dir}/{matcher}_inliers.csv"
        df.to_csv(csv_path, index=False)
        print(f"✅ CSV Arricchito salvato: {csv_path}")

# Esecuzione
enrich_csv_from_drive()


📊 Arricchimento dati per: SUPERPOINT-LG
✅ CSV Arricchito salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/superpoint-lg_inliers.csv

📊 Arricchimento dati per: LOFTR
✅ CSV Arricchito salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/loftr_inliers.csv

📊 Arricchimento dati per: SUPERGLUE
✅ CSV Arricchito salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/superglue_inliers.csv


## tokyo_xs

In [21]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import shutil

# --- 1. CONFIGURAZIONE PERCORSI ---
dataset_name = "tokyo_xs" # Cambiato in tokyo_xs
matchers = ["superpoint-lg", "loftr", "superglue"]
project_root = "/content/Visual-Place-Recognition-Project"
%cd {project_root}

# Cartelle su Drive per il salvataggio permanente (struttura dedicata a Tokyo)
drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset_name}"
torch_drive_dir = f"{drive_base}/torch_files"

os.makedirs(torch_drive_dir, exist_ok=True)

# --- 2. ESECUZIONE BATCH TOKYO-XS ---
for matcher in matchers:
    print(f"\n{'-'*50}\n🔥 MATCHING CORRENTE: {matcher.upper()} (Dataset: {dataset_name})\n{'-'*50}")

    preds_dir = f"logs/netvlad_{dataset_name}/predictions_txt"
    inliers_local_dir = f"{preds_dir}_{matcher}"

    # A. Esecuzione Matching (512x512, top 20 candidati)
    # Lo script salta automaticamente se riprendi una sessione interrotta
    !python match_queries_preds.py \
        --preds-dir "$preds_dir" \
        --matcher "$matcher" \
        --device cuda \
        --im-size 512 \
        --num-preds 20

    # B. Backup dei .torch su Drive (Salvataggio permanente)
    print(f"📦 Backup file .torch su Drive...")
    dest_torch = f"{torch_drive_dir}/{matcher}"

    # Se esiste già una cartella parziale, la aggiorniamo
    if os.path.exists(dest_torch):
        shutil.rmtree(dest_torch)
    shutil.copytree(inliers_local_dir, dest_torch)
    print(f"✅ Backup completato in: {dest_torch}")

    # C. Esecuzione Reranking (Calcolo Recall base per Tabella 1)
    print(f"📈 Calcolo Recall base per {matcher}...")
    !python reranking.py \
        --preds-dir "$preds_dir" \
        --inliers-dir "$inliers_local_dir" \
        --num-preds 20

print(f"\n{'-'*50}\n🎉 PROCESSO COMPLETATO PER TOKYO-XS\n{'-'*50}")

/content/Visual-Place-Recognition-Project

--------------------------------------------------
🔥 MATCHING CORRENTE: SUPERPOINT-LG (Dataset: tokyo_xs)
--------------------------------------------------
100% 315/315 [00:00<00:00, 65846.29it/s]
📦 Backup file .torch su Drive...
✅ Backup completato in: /content/drive/MyDrive/VPR_Project_Final_Results/tokyo_xs/torch_files/superpoint-lg
📈 Calcolo Recall base per superpoint-lg...
100% 315/315 [00:01<00:00, 172.80it/s]
R@1: 68.3, R@5: 72.1, R@10: 73.7, R@20: 78.7, R@100: 78.7

--------------------------------------------------
🔥 MATCHING CORRENTE: LOFTR (Dataset: tokyo_xs)
--------------------------------------------------
100% 315/315 [00:00<00:00, 66033.87it/s]
📦 Backup file .torch su Drive...
✅ Backup completato in: /content/drive/MyDrive/VPR_Project_Final_Results/tokyo_xs/torch_files/loftr
📈 Calcolo Recall base per loftr...
100% 315/315 [00:00<00:00, 479.31it/s]
R@1: 68.3, R@5: 72.7, R@10: 73.7, R@20: 78.7, R@100: 78.7

---------------------

### Generazione File CSV corretto

In [22]:
import os
import torch
import pandas as pd
from pathlib import Path
from util import get_list_distances_from_preds

# --- 1. CONFIGURAZIONE ---
# Puoi aggiungere altri nomi alla lista se hai già finito anche SVOX
datasets = ["sf_xs", "tokyo_xs"]
matchers = ["superpoint-lg", "loftr", "superglue"]
threshold = 25  # Soglia di 25 metri definita dal progetto

def generate_final_csvs():
    for dataset in datasets:
        print(f"\n{'='*60}\n📊 GENERAZIONE CSV PER DATASET: {dataset.upper()}\n{'='*60}")

        # Percorsi sul Drive
        drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset}"
        csv_drive_dir = f"{drive_base}/CSVs"
        torch_drive_dir = f"{drive_base}/torch_files"

        # Percorso dei TXT locali (necessari per le distanze GPS)
        preds_txt_dir = f"/content/Visual-Place-Recognition-Project/logs/netvlad_{dataset}/predictions_txt"

        os.makedirs(csv_drive_dir, exist_ok=True)

        for matcher in matchers:
            matcher_torch_path = Path(f"{torch_drive_dir}/{matcher}")

            if not matcher_torch_path.exists():
                print(f"⚠️ Cartella torch non trovata per {matcher} in {dataset}. Salto...")
                continue

            print(f"🔄 Elaborazione {matcher}...")
            data_list = []

            # Prendiamo tutti i file .torch salvati su Drive
            torch_files = sorted(list(matcher_torch_path.glob("*.torch")), key=lambda x: int(x.stem))

            for t_file in torch_files:
                # 1. Recupero distanze reali dal file TXT locale
                txt_file = f"{preds_txt_dir}/{t_file.stem}.txt"
                if not os.path.exists(txt_file):
                    continue
                geo_dists = get_list_distances_from_preds(txt_file)

                # 2. Carico i risultati del matching dal Drive
                query_results = torch.load(t_file, weights_only=False)

                # 3. Estraggo gli inliers per tutti i 20 candidati
                inliers_counts = [res['num_inliers'] for res in query_results]

                # 4. Calcolo il MASSIMO degli inliers (confidenza del miglior match)
                max_val = max(inliers_counts) if inliers_counts else 0

                # 5. Determino se il miglior match (quello con più inliers) è corretto
                # Se più immagini hanno lo stesso numero di inliers, prendiamo la prima (top-1)
                best_idx = inliers_counts.index(max_val) if inliers_counts else 0
                is_correct = 1 if (geo_dists[best_idx] <= threshold) else 0

                data_list.append({
                    'query_id': t_file.stem,
                    'max_inliers': max_val,
                    'is_correct': is_correct
                })

            # Salvataggio del CSV finale su Drive
            df = pd.DataFrame(data_list)
            csv_name = f"{matcher}_stats_final.csv"
            csv_path = f"{csv_drive_dir}/{csv_name}"
            df.to_csv(csv_path, index=False)
            print(f"✅ CSV salvato: {csv_path} ({len(df)} query)")

# Esecuzione dello script
generate_final_csvs()


📊 GENERAZIONE CSV PER DATASET: SF_XS
🔄 Elaborazione superpoint-lg...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/superpoint-lg_stats_final.csv (1000 query)
🔄 Elaborazione loftr...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/loftr_stats_final.csv (1000 query)
🔄 Elaborazione superglue...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/sf_xs/CSVs/superglue_stats_final.csv (1000 query)

📊 GENERAZIONE CSV PER DATASET: TOKYO_XS
🔄 Elaborazione superpoint-lg...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/tokyo_xs/CSVs/superpoint-lg_stats_final.csv (315 query)
🔄 Elaborazione loftr...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/tokyo_xs/CSVs/loftr_stats_final.csv (315 query)
🔄 Elaborazione superglue...
✅ CSV salvato: /content/drive/MyDrive/VPR_Project_Final_Results/tokyo_xs/CSVs/superglue_stats_final.csv (315 query)


## svox (sun e night)

In [23]:
import os
import sys
import torch
import pandas as pd
import numpy as np
from pathlib import Path
import shutil

# --- 1. CONFIGURAZIONE ---
# Gestiamo entrambi i set di SVOX in un colpo solo
svox_datasets = ["svox_sun", "svox_night"]
matchers = ["superpoint-lg", "loftr", "superglue"]
project_root = "/content/Visual-Place-Recognition-Project"
%cd {project_root}

for dataset_name in svox_datasets:
    print(f"\n{'#'*60}\n🌟 INIZIO ELABORAZIONE DATASET: {dataset_name.upper()}\n{'#'*60}")

    # Cartelle su Drive per il salvataggio permanente
    drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset_name}"
    torch_drive_dir = f"{drive_base}/torch_files"
    os.makedirs(torch_drive_dir, exist_ok=True)

    for matcher in matchers:
        print(f"\n{'-'*50}\n🔥 MATCHING CORRENTE: {matcher.upper()} ({dataset_name})\n{'-'*50}")

        preds_dir = f"logs/netvlad_{dataset_name}/predictions_txt"
        inliers_local_dir = f"{preds_dir}_{matcher}"

        # A. Esecuzione Matching (512x512, top 20)
        !python match_queries_preds.py \
            --preds-dir "$preds_dir" \
            --matcher "$matcher" \
            --device cuda \
            --im-size 512 \
            --num-preds 20

        # B. Backup dei .torch su Drive
        print(f"📦 Backup file .torch su Drive...")
        dest_torch = f"{torch_drive_dir}/{matcher}"
        if os.path.exists(dest_torch):
            shutil.rmtree(dest_torch)
        shutil.copytree(inliers_local_dir, dest_torch)
        print(f"✅ Backup completato in: {dest_torch}")

        # C. Esecuzione Reranking (Recall base)
        print(f"📈 Calcolo Recall base per {matcher}...")
        !python reranking.py \
            --preds-dir "$preds_dir" \
            --inliers-dir "$inliers_local_dir" \
            --num-preds 20

print(f"\n{'#'*60}\n🎉 TUTTI I DATASET SVOX SONO COMPLETATI!\n{'#'*60}")

/content/Visual-Place-Recognition-Project

############################################################
🌟 INIZIO ELABORAZIONE DATASET: SVOX_SUN
############################################################

--------------------------------------------------
🔥 MATCHING CORRENTE: SUPERPOINT-LG (svox_sun)
--------------------------------------------------
100% 712/712 [42:36<00:00,  3.59s/it]
📦 Backup file .torch su Drive...
✅ Backup completato in: /content/drive/MyDrive/VPR_Project_Final_Results/svox_sun/torch_files/superpoint-lg
📈 Calcolo Recall base per superpoint-lg...
100% 712/712 [00:05<00:00, 125.09it/s]
R@1: 47.8, R@5: 49.6, R@10: 50.6, R@20: 51.4, R@100: 51.4

--------------------------------------------------
🔥 MATCHING CORRENTE: LOFTR (svox_sun)
--------------------------------------------------
100% 712/712 [44:11<00:00,  3.72s/it]
📦 Backup file .torch su Drive...
✅ Backup completato in: /content/drive/MyDrive/VPR_Project_Final_Results/svox_sun/torch_files/loftr
📈 Calcolo Recal

### Generazione File CSV

In [24]:
import os
import torch
import pandas as pd
from pathlib import Path
from util import get_list_distances_from_preds

# --- 1. CONFIGURAZIONE ---
# Specifichiamo i due sottoset di SVOX
datasets = ["svox_sun", "svox_night"]
matchers = ["superpoint-lg", "loftr", "superglue"]
threshold = 25  # Soglia di 25 metri richiesta dal progetto

def generate_svox_final_csvs():
    for dataset in datasets:
        print(f"\n{'='*60}\n📊 GENERAZIONE CSV PER DATASET: {dataset.upper()}\n{'='*60}")

        # Percorsi sul Drive (dove sono stati salvati i .torch nel passo precedente)
        drive_base = f"/content/drive/MyDrive/VPR_Project_Final_Results/{dataset}"
        csv_drive_dir = f"{drive_base}/CSVs"
        torch_drive_dir = f"{drive_base}/torch_files"

        # Percorso dei TXT locali (necessari per calcolare is_correct tramite distanze GPS)
        preds_txt_dir = f"/content/Visual-Place-Recognition-Project/logs/netvlad_{dataset}/predictions_txt"

        os.makedirs(csv_drive_dir, exist_ok=True)

        for matcher in matchers:
            matcher_torch_path = Path(f"{torch_drive_dir}/{matcher}")

            if not matcher_torch_path.exists():
                print(f"⚠️ Cartella torch non trovata per {matcher} in {dataset}. Salto...")
                continue

            print(f"🔄 Elaborazione {matcher}...")
            data_list = []

            # Recuperiamo tutti i file .torch dal Drive
            torch_files = sorted(list(matcher_torch_path.glob("*.torch")), key=lambda x: int(x.stem))

            for t_file in torch_files:
                # 1. Recupero distanze reali (Ground Truth) dal file TXT corrispondente
                txt_file = f"{preds_txt_dir}/{t_file.stem}.txt"
                if not os.path.exists(txt_file):
                    continue
                geo_dists = get_list_distances_from_preds(txt_file)

                # 2. Carico i tensori degli inliers dal Drive
                query_results = torch.load(t_file, weights_only=False)

                # 3. Estraggo il numero di inliers per i 20 candidati
                inliers_counts = [res['num_inliers'] for res in query_results]

                # 4. Trovo il massimo numero di inliers (confidenza della predizione)
                max_val = max(inliers_counts) if inliers_counts else 0

                # 5. Determino se il miglior match dopo il re-ranking è corretto
                best_idx = inliers_counts.index(max_val) if inliers_counts else 0
                is_correct = 1 if (geo_dists[best_idx] <= threshold) else 0

                data_list.append({
                    'query_id': t_file.stem,
                    'max_inliers': max_val,
                    'is_correct': is_correct
                })

            # Creazione del DataFrame e salvataggio permanente su Drive
            df = pd.DataFrame(data_list)
            csv_name = f"{matcher}_stats_final.csv"
            csv_path = f"{csv_drive_dir}/{csv_name}"
            df.to_csv(csv_path, index=False)
            print(f"✅ CSV Statistiche salvato: {csv_path} ({len(df)} query)")

# Esecuzione dello script di arricchimento
generate_svox_final_csvs()


📊 GENERAZIONE CSV PER DATASET: SVOX_SUN
🔄 Elaborazione superpoint-lg...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_sun/CSVs/superpoint-lg_stats_final.csv (712 query)
🔄 Elaborazione loftr...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_sun/CSVs/loftr_stats_final.csv (712 query)
🔄 Elaborazione superglue...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_sun/CSVs/superglue_stats_final.csv (712 query)

📊 GENERAZIONE CSV PER DATASET: SVOX_NIGHT
🔄 Elaborazione superpoint-lg...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_night/CSVs/superpoint-lg_stats_final.csv (702 query)
🔄 Elaborazione loftr...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_night/CSVs/loftr_stats_final.csv (702 query)
🔄 Elaborazione superglue...
✅ CSV Statistiche salvato: /content/drive/MyDrive/VPR_Project_Final_Results/svox_night/CSVs/superglue_st

# Analisi delle Performance in Visual Place Recognition: Matching Geometrico, Strategie Adattive e Stima dell'Incertezza

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from sklearn.metrics import precision_recall_curve, auc

# --- CONFIGURAZIONE PERCORSI ---
drive_root = "/content/drive/MyDrive/VPR_Project_Final_Results"
output_dir = "final_plots"
os.makedirs(output_dir, exist_ok=True)

# Lista dei dataset e matcher che abbiamo elaborato
datasets = ["sf_xs", "tokyo_xs", "svox_sun", "svox_night"]
matchers = ["superpoint-lg", "loftr", "superglue"]

def analyze_vpr_performance(csv_path, dataset_name, matcher_name):
    if not os.path.exists(csv_path):
        return

    df = pd.read_csv(csv_path)
    inlier_col = 'max_inliers'
    correct_col = 'is_correct'

    print(f"📈 Analisi in corso: {dataset_name} + {matcher_name}")

    # --- 1. PLOT 6.1: ADAPTIVE TRADE-OFF (Recall vs Cost Saving) ---
    # Il Cost Saving qui è calcolato come quante query NON avrebbero bisogno
    # di re-ranking se usassimo la soglia tau.
    max_val = int(df[inlier_col].max())
    thresholds = np.arange(0, max_val + 1, 1)
    recalls = []
    cost_savings = []
    total_q = len(df)

    for t in thresholds:
        # Se inliers >= t, consideriamo la query "EASY" (niente re-ranking necessario)
        # Se inliers < t, facciamo re-ranking.
        # Qui simuliamo l'impatto sulla Recall finale.
        correct_above_t = df[(df[correct_col] == 1) & (df[inlier_col] >= t)].shape[0]
        recalls.append((correct_above_t / total_q) * 100)

        # Più alta è la soglia, meno query consideriamo "EASY", meno risparmio abbiamo
        easy_queries = df[df[inlier_col] >= t].shape[0]
        cost_savings.append((easy_queries / total_q) * 100)

    fig, ax1 = plt.subplots(figsize=(10, 5))
    ax1.plot(thresholds, recalls, color='tab:blue', label='Recall@1 (Adaptive)', linewidth=2)
    ax1.set_xlabel('Inlier Threshold (τ)')
    ax1.set_ylabel('Recall@1 (%)', color='tab:blue')
    ax2 = ax1.twinx()
    ax2.plot(thresholds, cost_savings, color='tab:green', linestyle='--', label='Cost Saving (%)')
    ax2.set_ylabel('Potential Computational Saving (%)', color='tab:green')
    plt.title(f'6.1 Adaptive Re-ranking: {dataset_name} ({matcher_name})')
    plt.savefig(f"{output_dir}/{dataset_name}_{matcher_name}_6.1_adaptive.png")
    plt.close()

    # --- 2. PLOT 6.2: UNCERTAINTY (PR Curve per AUPRC) ---
    # L'obiettivo è vedere se "max_inliers" predice bene se una query è sbagliata (is_correct=0)
    # Invertiamo gli inliers perché meno inliers = più probabilità di errore (incertezza)
    uncertainty_score = -df[inlier_col]
    y_true = 1 - df[correct_col] # 1 indica "Errore" (classe positiva per l'incertezza)

    precision, recall, _ = precision_recall_curve(y_true, uncertainty_score)
    auprc_score = auc(recall, precision)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='darkorange', label=f'PR Curve (AUPRC = {auprc_score:.3f})')
    plt.xlabel('Recall (Detection of Errors)')
    plt.ylabel('Precision (Confidence in Error Prediction)')
    plt.title(f'6.2 Uncertainty Estimation: {dataset_name} ({matcher_name})')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.savefig(f"{output_dir}/{dataset_name}_{matcher_name}_6.2_auprc.png")
    plt.close()

    return auprc_score

# Esecuzione per tutti i file trovati su Drive
summary_results = []
for ds in datasets:
    for mt in matchers:
        csv_p = f"{drive_root}/{ds}/CSVs/{mt}_stats_final.csv"
        score = analyze_vpr_performance(csv_p, ds, mt)
        if score:
            summary_results.append({'Dataset': ds, 'Matcher': mt, 'AUPRC': score})

# --- 3. TABELLA RIASSUNTIVA ---
df_summary = pd.DataFrame(summary_results)
print("\n--- RISULTATI FINALI PER REPORT ---")
print(df_summary)
df_summary.to_csv(f"{output_dir}/summary_metrics.csv", index=False)

📈 Analisi in corso: sf_xs + superpoint-lg
📈 Analisi in corso: sf_xs + loftr
📈 Analisi in corso: sf_xs + superglue
📈 Analisi in corso: tokyo_xs + superpoint-lg
📈 Analisi in corso: tokyo_xs + loftr
📈 Analisi in corso: tokyo_xs + superglue
📈 Analisi in corso: svox_sun + superpoint-lg
📈 Analisi in corso: svox_sun + loftr
📈 Analisi in corso: svox_sun + superglue
📈 Analisi in corso: svox_night + superpoint-lg
📈 Analisi in corso: svox_night + loftr
📈 Analisi in corso: svox_night + superglue

--- RISULTATI FINALI PER REPORT ---
       Dataset        Matcher     AUPRC
0        sf_xs  superpoint-lg  0.958964
1        sf_xs          loftr  0.955359
2        sf_xs      superglue  0.955096
3     tokyo_xs  superpoint-lg  0.966857
4     tokyo_xs          loftr  0.960619
5     tokyo_xs      superglue  0.919419
6     svox_sun  superpoint-lg  0.978083
7     svox_sun          loftr  0.977251
8     svox_sun      superglue  0.974188
9   svox_night  superpoint-lg  0.993302
10  svox_night          loftr  0.9

Zip e download cartella final_plot

In [26]:
!zip -r final_plots.zip final_plots/

  adding: final_plots/ (stored 0%)
  adding: final_plots/svox_night_superpoint-lg_6.2_auprc.png (deflated 15%)
  adding: final_plots/tokyo_xs_superpoint-lg_6.1_adaptive.png (deflated 6%)
  adding: final_plots/sf_xs_superglue_6.1_adaptive.png (deflated 7%)
  adding: final_plots/svox_night_loftr_6.1_adaptive.png (deflated 7%)
  adding: final_plots/svox_sun_loftr_6.1_adaptive.png (deflated 6%)
  adding: final_plots/svox_sun_superpoint-lg_6.2_auprc.png (deflated 15%)
  adding: final_plots/sf_xs_loftr_6.2_auprc.png (deflated 14%)
  adding: final_plots/svox_night_superglue_6.1_adaptive.png (deflated 6%)
  adding: final_plots/svox_night_superglue_6.2_auprc.png (deflated 15%)
  adding: final_plots/sf_xs_superpoint-lg_6.1_adaptive.png (deflated 6%)
  adding: final_plots/tokyo_xs_loftr_6.1_adaptive.png (deflated 6%)
  adding: final_plots/svox_sun_superpoint-lg_6.1_adaptive.png (deflated 6%)
  adding: final_plots/svox_sun_superglue_6.2_auprc.png (deflated 14%)
  adding: final_plots/sf_xs_loftr_6.

In [27]:
from google.colab import files
files.download('final_plots.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>